In [1]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams
from packages.Model.CFNet import CFNet3

Using TensorFlow backend.


# Inisialisasi / Persiapan

In [2]:
fold_index = 0
dataset_dir = 'dataset/bin/CF.32.B.30K'
trained_dir = 'trained/CF.32.B.30K CF3.1'

# Memuat Dataset

In [3]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [4]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Paremeter Gabor

In [5]:
class ChannelizeGaborParams(GaborParams):
    
    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        
        i += 1
        j += 1
        
        lambd = (i * j) / shape[2]
        theta = degree * np.pi / 180
        psi = (i + j) / shape[2]
        sigma = (i + j)
        gamma = (i + j) / shape[2]
        
        return lambd, theta, psi, sigma, gamma

# Mendefinisikan Kernel

In [6]:
kernel1 = GaborFilterBanks(ChannelizeGaborParams())
kernel2 = GaborFilterBanks(ChannelizeGaborParams())
kernel3 = GaborFilterBanks(ChannelizeGaborParams())
kernel4 = GaborFilterBanks(ChannelizeGaborParams())
kernel5 = GaborFilterBanks(ChannelizeGaborParams())
kernel6 = GaborFilterBanks(ChannelizeGaborParams())

# Mendefinisikan Model

In [9]:
ars = CFNet3(kernels = {
    1: kernel1,
    2: kernel2,
    3: kernel3,
    4: kernel4,
    5: kernel5,
    6: kernel6
})
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')

model = ars.model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
c2 (Conv2D)                  (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
c3 (Conv2D)                  (None, 16, 16, 64)        18496     
_________________________________________________________________
c4 (Conv2D)                  (None, 16, 16, 64)        36928     
_________________________________________________________________
mp4 (MaxPooling2D)           (None, 8, 8, 64)          0         
_________________________________________________________________
c5 (Conv2D)                  (None, 8, 8, 128)         73856     
__________

# Pelatihan

In [ ]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 100, verbose = 1, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch = 0)

Epoch 1/100
22500/22500 [==============================] - 143s 6ms/step - loss: 2.1670 - acc: 0.2031
Accuracy : 0.26004444444444447
Model saved to disk
Passes : 1/50
Epoch 2/100
22500/22500 [==============================] - 154s 7ms/step - loss: 2.0332 - acc: 0.2637
Accuracy : 0.22857777777777777
Model saved to disk
Passes : 2/50
Epoch 3/100
22500/22500 [==============================] - 159s 7ms/step - loss: 1.9794 - acc: 0.2887
Accuracy : 0.3022666666666667
Model saved to disk
Passes : 3/50
Epoch 4/100
22500/22500 [==============================] - 155s 7ms/step - loss: 1.9464 - acc: 0.2996
Accuracy : 0.3206222222222222
Model saved to disk
Passes : 4/50
Epoch 5/100
22500/22500 [==============================] - 157s 7ms/step - loss: 1.9316 - acc: 0.3087
Accuracy : 0.2858222222222222
Model saved to disk
Passes : 5/50
Epoch 6/100
22500/22500 [==============================] - 155s 7ms/step - loss: 1.9121 - acc: 0.3152
Accuracy : 0.3072888888888889
Model saved to disk
Passes : 6/50
Ep